In [12]:
import numpy as np 
from utils import *
from haar_like_features import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve
from skimage import io
from skimage.color import rgb2gray
from skimage.transform import resize
import matplotlib.pyplot as plt
import pickle
import os
import time
import cv2

utils = Utils()
haar = HaarLikeFeatures()

%matplotlib inline
%reload_ext autoreload
%autoreload 2

## Extract Labels


In [13]:
features_values = haar.extract_features_as_labels(0, 0, 19, 19)
utils.save_pickle(features_values, 'features_values.pkl')

In [15]:
features_values[0].shape

(5,)

## Prepare Data


## Weak Classifier


In [ ]:
class WeakClassifier:
    """
    theta: threshold
    feature: feature value
            numpy array of shape (5, )
    polarity: polarity

    """
    def __init__(self, feature_label, feature, theta, polarity):
        self.feature_label = feature_label
        self.feature = feature
        self.theta = theta
        self.polarity = polarity
        self.haar = HaarLikeFeatures()

    def predict_using_intgeral_images(self, X):
        """
        X: input of integral images
           numpy array of shape (N, 19, 19)
        """
        features = [self.haar.get_feautures_result(i, [self.feature])[0] for i in X] 
        y_pred = np.zeros((X.shape[0], 1))
        y_pred[self.polarity * features < self.polarity * self.theta] = 1
        return y_pred
    
    def pred_using_values(self, X):
        """
        X: input of features
           numpy array of shape (N, 5)
        """
        y_pred = np.zeros((X.shape[0], 1))
        y_pred[self.polarity * X < self.polarity * self.theta] = 1
        return y_pred
    
    def __str__(self):
        return f'WeakClassifier(p={self.p}, theta={self.theta}, polarity={self.polarity})'

In [ ]:
class viola_jones_classifier:
    def train(self, X, y):
        # ============= initalizing training =============="
        P = X[y == 1]
        N = X[y == 0]
        self.w = np.ones_like(y)

        wn = 1 / (2*len(N))
        wp = 1 / (2*len(P))
        self.w[y == 1] = wp
        self.w[y == 0] = wn
        # ============= initalizing training =============="

        # ============= start training =============="
        for t in range(self.T):

            print(" Training Weak Classifier: ", t)
            self.w = self.w / np.sum(self.w)
            total_positive = np.sum(self.w[y == 1])
            total_negative = np.sum(self.w[y == 0])

            weak_clf = []
            for i, features in enumerate(X):
                sorted_features = np.argsort(features)
